In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

load_url = "https://www.ski-gelende.com/gelende/"
html = requests.get(load_url)
soup = BeautifulSoup(html.content, "html.parser")

html = str(soup)
urls = []
names = []
cnt = 3
while True:
    try:
        url = re.search( r'(?<=<li><a href=").+?(?=">)', html).group(0)
        urls.append(url)
        
        index = html.find("<li><a href=\""+url+"\">")
        html = html[index:]
        html = html.replace('<li><a href=\"'+url+'\"','')
        
        name = re.search( r'(?<=>).+?(?=</a>)', html).group(0)
        names.append(name)
        if name == "箱館山スキー場":
            break
    except:
        
        break


url_list = []

for i in range(len(names)):
  # Google検索するキーワードを設定
  search_word = names[i]

  # 上位から何件までのサイトを抽出するか指定する
  pages_num = 1 + 1

  print(f'【検索ワード】{search_word}')

  # Googleから検索結果ページを取得する
  url = f'https://www.google.co.jp/search?hl=ja&num={pages_num}&q={search_word}'
  request = requests.get(url)

  # Googleのページ解析を行う
  soup = BeautifulSoup(request.text, "html.parser")
  search_site_list = soup.select('div.kCrYT > a')

  # ページ解析と結果の出力
  for rank, site in zip(range(1, pages_num), search_site_list):
      try:
          site_title = site.select('h3.zBAuLc')[0].text
      except IndexError:
          site_title = site.select('img')[0]['alt']
      site_url = site['href'].replace('/url?q=', '')
      # 結果を出力する
      target1 = 'http'
      idx1 = site_url.find(target1)
      target2 = '&sa=U'
      idx2 = site_url.find(target2)
      site_url = site_url[idx1 + len(target1):idx2]
      site_url = 'http' + site_url
      url_list.append(site_url)

print(url_list)